In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [2]:
#path_df = "Review_dataset.pickle"

In [3]:
#with open(path_df, 'rb') as data:
df = pd.read_csv("../Final Project/top5.csv")

In [4]:
df.head()

,Reviewer_Nationality,Full_Review
0,Ireland,No real complaints the hotel was great great ...
1,Australia,Location was good and staff were ok It is cut...
2,UnitedKingdom,Great location in nice surroundings the bar a...
3,UnitedKingdom,The room is spacious and bright The hotel is ...
4,UnitedKingdom,Good location Set in a lovely park friendly s...


In [5]:
#1. Text cleaning and preparation
# \r and \n
df['Content_Parsed_1'] = df['Full_Review'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")

In [12]:
# " when quoting text
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("No Positive", '')
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("No Negative", '')

In [13]:
# " when quoting text
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

In [14]:
# Lowercasing the text
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

In [15]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

In [16]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

In [17]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jcaravella\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jcaravella\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In [19]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [20]:
df['Content_Parsed_5'] = lemmatized_text_list

In [21]:
##Stop Words# Downloading the stop words list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jcaravella\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
# Loading the stop words in english
stop_words = list(stopwords.words('english'))

In [23]:
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [24]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

In [25]:
df.head(3)

,Reviewer_Nationality,Full_Review,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,Ireland,No real complaints the hotel was great great ...,No real complaints the hotel was great great ...,no real complaints the hotel was great great ...,no real complaints the hotel was great great ...,no real complaints the hotel was great great ...,no real complaints the hotel be great great l...,real complaints hotel great great location...
1,Australia,Location was good and staff were ok It is cut...,Location was good and staff were ok It is cut...,location was good and staff were ok it is cut...,location was good and staff were ok it is cut...,location was good and staff were ok it is cut...,location be good and staff be ok it be cute h...,location good staff ok cute hotel break...
2,UnitedKingdom,Great location in nice surroundings the bar a...,Great location in nice surroundings the bar a...,great location in nice surroundings the bar a...,great location in nice surroundings the bar a...,great location in nice surroundings the bar a...,great location in nice surround the bar and r...,great location nice surround bar restauran...


In [26]:
list_columns = ["Reviewer_Nationality", "Full_Review", "Content_Parsed_6"]
df = df[list_columns]
df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})
df.head()

,Reviewer_Nationality,Full_Review,Content_Parsed
0,Ireland,No real complaints the hotel was great great ...,real complaints hotel great great location...
1,Australia,Location was good and staff were ok It is cut...,location good staff ok cute hotel break...
2,UnitedKingdom,Great location in nice surroundings the bar a...,great location nice surround bar restauran...
3,UnitedKingdom,The room is spacious and bright The hotel is ...,room spacious bright hotel locate quie...
4,UnitedKingdom,Good location Set in a lovely park friendly s...,good location set lovely park friendly staf...


In [27]:
df.to_csv(r'C:\Users\jcaravella\Documents\Final Project\contentparsed.csv', index = False,header=True)